In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## LogSoftmax说明

例如我们的输入是(1,1,1,1), 那么LogSoftmax就是进行如下的操作.

$$
Log(\frac{e^{1}}{e^{1}+e^{1}+e^{1}+e^{1}}) = -1.3863
$$

我们拆开来看就是首先计算Softmax

$$
\frac{e^{1}}{e^{1}+e^{1}+e^{1}+e^{1}} = 0.25
$$

接着对上面的结果取对数

$$
Log(0.25) = -1.3863
$$

In [33]:
ll = nn.LogSoftmax()
input = torch.tensor([1.0,1.0,1.0,1.0])
output = ll(input)
print(output)

tensor([-1.3863, -1.3863, -1.3863, -1.3863])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  This is separate from the ipykernel package so we can avoid doing imports until


In [34]:
torch.log(torch.tensor([0.25]))

tensor([-1.3863])

## NLLLoss说明

下面的例子中, target为2, 也就是表示one-hot表示为(0,0,1), 于是NLLLoss的计算如下所示.

$$
-(0*0.2+0*0.3+1*0.5)=-0.5
$$

In [35]:
loss = nn.NLLLoss()
input = torch.tensor([[0.2, 0.3, 0.5]])
target = torch.tensor([2]).long()
output = loss(input, target)
print(output)

tensor(-0.5000)


## CrossEntropy说明

首先我们先不管Pytorch中是如何实现交叉熵的, 我们先自己来看一下交叉熵是如何计算的.

交叉熵的计算公式如下所示:

$$
Loss = \hat{-y_{i}}*Log(y_{i})
$$

其中,

- $\hat{-y_{i}}$是实际值
- $y_{i}$是预测值

假设现在的predict为$(\frac{1}{4}, \frac{1}{4}, \frac{1}{4}, \frac{1}{4})$, target为$(0, 0, 0, 1)$, 那么此时的交叉熵就是:

$$
-1*Log(\frac{1}{4}) = 1.3863
$$

---

下面我们看一下PyTorch中CrossEntropy是如何计算的. 前面说了, CrossEntropy是LogSoftmax和NLLLoss的结合.

比如此时output为$(a_{1}, a_{2}, a_{3}, a_{4})$, 最后的target是${0,1,0,0}$.

首先output经过LogSoftmax之后变为下面的式子.

$$
(log(\frac{e^{a_{1}}}{\sum_{i=1}^{4}{a_{i}}}), log(\frac{e^{a_{2}}}{\sum_{i=1}^{4}{a_{i}}}), log(\frac{e^{a_{3}}}{\sum_{i=1}^{4}{a_{i}}}), log(\frac{e^{a_{4}}}{\sum_{i=1}^{4}{a_{i}}}))
$$

接着计算NLLLoss

$$
-1*log(\frac{e^{a_{2}}}{\sum_{i=1}^{4}{a_{i}}}) = -log(\frac{e^{a_{2}}}{\sum_{i=1}^{4}{a_{i}}})
$$

这个结果是和上面交叉熵的定义是一样的. 我们可以想象成在LogSoftmax中, 我们不仅将原来的output转换为了概率值, 还求了log, 最后只需要和target相乘即可.

----

我们看一下自己上面的过程是不是和计算的结果是一样的. 只需要将CrossEntropyLoss的input设置为(1,1,1,1), 这样经过softmax之后的概率就$(\frac{1}{4}, \frac{1}{4}, \frac{1}{4}, \frac{1}{4})$, 所以这里计算得到的交叉熵应该是1.3863.

下面实际运算一下

In [36]:
loss = nn.CrossEntropyLoss()
input = torch.tensor([[1, 1, 1.0, 1]])
target = torch.tensor([2]).long()
output = loss(input, target)
print(output)

tensor(1.3863)


我们知道

- 当预测的概率为(1,0,0), 实际就是(1,0,0)的时候, 交叉熵是接近0的.
- 当预测的概率为(0,0,1), 实际就是(1,0,0)的时候, 交叉熵是接近正无穷的.

下面看一下例子.

In [37]:
# 预测和实际很接近
loss = nn.CrossEntropyLoss()
input = torch.tensor([[0, 0, 100.0]])
target = torch.tensor([2]).long()
output = loss(input, target)
print(output)

tensor(0.)


In [41]:
# 预测和实际差很远
loss = nn.CrossEntropyLoss()
input = torch.tensor([[100.0, 0, 0]])
target = torch.tensor([2]).long()
output = loss(input, target)
print(output)

tensor(100.)
